In [61]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [62]:
from pyprojroot import here
import os
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import shap

path_data = here("./data")
os.chdir(path_data)

data = pd.read_csv("data_er_visits.csv").rename(columns={"Unnamed: 0": "Patient ID"})
data.head()

,Patient ID,Hospital ID,Age 60+,High Cholesterol,Diabetes,Preventative Services,ER Visit
0,0,1,0,0,0,1,0
1,1,1,1,0,0,0,1
2,2,3,1,0,0,1,0
3,3,1,0,1,0,0,0
4,4,3,0,0,0,1,0


In [63]:
# Prepare the data
X = data.drop(["ER Visit", "Hospital ID"], axis=1)
y = data["ER Visit"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train the XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

# Make predictions (opxtional, to evaluate model)
predictions = model.predict(X_test)

In [64]:
# Assuming 'data' is your DataFrame and is already defined
# ... [Your existing code for data preparation and model training] ...

# Create a Tree explainer
explainer = shap.Explainer(
    model, X_train, model_output="probability", feature_perturbation="interventional"
)

# Calculate SHAP values - this might take some time for larger datasets
shap_values = explainer(data.drop(["ER Visit", "Hospital ID"], axis=1))

 96%|=================== | 9631/10000 [00:23<00:00]       

In [65]:
# Assuming shap_values and X have the same order
column_mapping = {
    i: name
    for i, name in enumerate(data.drop(["ER Visit", "Hospital ID"], axis=1).columns)
}

# Rename columns in the DataFrame created from SHAP values
data_shap_pd = pd.DataFrame(shap_values.values).rename(columns=column_mapping)

In [69]:
data_no_id_outcome = data.drop(columns=["Patient ID", "Hospital ID", "ER Visit"])
data_percentile = data_no_id_outcome.rank(pct=True)

In [70]:
data_percentile

,Age 60+,High Cholesterol,Diabetes,Preventative Services
0,0.3557,0.2973,0.3777,0.75005
1,0.8557,0.2973,0.3777,0.25005
2,0.8557,0.2973,0.3777,0.75005
3,0.3557,0.7973,0.3777,0.25005
4,0.3557,0.2973,0.3777,0.75005
...,...,...,...,...
9995,0.8557,0.7973,0.8777,0.75005
9996,0.8557,0.2973,0.3777,0.75005
9997,0.8557,0.2973,0.3777,0.25005
9998,0.3557,0.2973,0.3777,0.25005


Step: Add back hosptial ids and add unique ids

In [83]:
data_percentile_id = pd.concat(
    [data[["Patient ID", "Hospital ID"]], data_percentile], axis=1
)
data_shap_id = pd.concat([data[["Hospital ID"]], data_shap_pd], axis=1)

In [84]:
import pandas as pd


def median_shap_for_high_percentiles(percentile_df, shap_df, id_col):
    # Initialize a dictionary to store the results
    median_shap_values = {id_col: [], "variable": [], "median_shap": []}

    # Iterate over each ID
    for id_value in percentile_df[id_col].unique():
        # Filter DataFrames for the current ID
        percentile_subdf = percentile_df[percentile_df[id_col] == id_value]
        shap_subdf = shap_df[shap_df[id_col] == id_value]

        # Iterate over each column (except the ID column)
        for col in percentile_df.columns:
            if col == id_col:
                continue

            # Calculate the median of the current column in percentile DataFrame
            median_value = percentile_subdf[col].median()

            # Filter rows where the percentile value is above the median
            rows_above_median = percentile_subdf[
                percentile_subdf[col] > median_value
            ].index

            # Calculate the median SHAP value for these rows
            median_shap = shap_subdf.loc[rows_above_median, col].median()

            # Store the results
            median_shap_values[id_col].append(id_value)
            median_shap_values["variable"].append(col)
            median_shap_values["median_shap"].append(median_shap)

    # Convert the dictionary to a DataFrame and return
    return pd.DataFrame(median_shap_values)

In [87]:
# Example usage
result_df = (
    median_shap_for_high_percentiles(
        percentile_df=data_percentile_id, shap_df=data_shap_id, id_col="Hospital ID"
    )
    .fillna(-0.20)
    .round(2)
)

In [91]:
noise = np.random.uniform(-0.07, 0.07, result_df["median_shap"].shape)
result_df["median_shap"] = result_df["median_shap"] + noise
data_shap_hospital = (
    result_df.rename(columns={"variable": "Driver", "median_shap": "Impact"})
    .round(2)
    .query("Driver != 'Patient ID' ")
).reset_index(drop=True)

,Hospital ID,Driver,Impact
0,1,Age 60+,0.32
1,1,High Cholesterol,0.23
2,1,Diabetes,0.18
3,1,Preventative Services,-0.18
4,3,Age 60+,0.42
5,3,High Cholesterol,0.33
6,3,Diabetes,0.24
7,3,Preventative Services,-0.23
8,2,Age 60+,0.44
9,2,High Cholesterol,0.19


In [92]:
data_shap_hospital.to_csv("data_shap_hospital.csv")

In [93]:
data_shap_patient_id = data_shap_id.reset_index().rename(columns={"index": "Member ID"})

data_shap_patient_id = data_shap_patient_id.round(2)
data_shap_patient_id

,Member ID,Hospital ID,Patient ID,Age 60+,High Cholesterol,Diabetes,Preventative Services
0,0,1,-0.04,-0.06,-0.06,-0.01,-0.12
1,1,1,0.01,0.43,-0.14,-0.02,0.24
2,2,3,-0.07,0.21,-0.10,-0.03,-0.26
3,3,1,-0.17,-0.15,0.11,-0.05,0.12
4,4,3,-0.04,-0.06,-0.06,-0.01,-0.12
...,...,...,...,...,...,...,...
9995,9995,2,0.03,0.35,0.29,0.20,-0.28
9996,9996,1,-0.03,0.17,-0.14,-0.03,-0.23
9997,9997,1,0.04,0.30,-0.24,-0.04,0.21
9998,9998,2,0.10,-0.11,-0.20,-0.04,0.19
